# Retinal Degeneration Associated With RPGRIP1


Data from [Beryozkin A, et al. Retinal Degeneration Associated With RPGRIP1: A Review of Natural History, Mutation Spectrum, and Genotype-Phenotype Correlation in 228 Patients](https://pubmed.ncbi.nlm.nih.gov/34722527)

In [1]:
import genophenocorr

print(f"Using genophenocorr version {genophenocorr.__version__}")

Using genophenocorr version 0.1.1dev


## Setup

### Load HPO

We use HPO `v2023-10-09` release for this analysis.

In [2]:
import hpotk

fpath_hpo = 'https://github.com/obophenotype/human-phenotype-ontology/releases/download/v2023-10-09/hp.json'
hpo = hpotk.load_minimal_ontology(fpath_hpo)

### Load Phenopackets

We will load phenopacket JSON files located in `phenopackets` folder that is next to the notebook.

In [3]:
from genophenocorr.preprocessing import configure_caching_patient_creator, load_phenopacket_folder

fpath_phenopackets = 'phenopackets'
pc = configure_caching_patient_creator(hpo)
cohort = load_phenopacket_folder(fpath_phenopackets, pc)

Patients Created:   0%|          | 0/229 [00:00<?, ?it/s]Expected a VCF record, a VRS CNV, or an expression with `hgvs.c` but did not find one in patient PMID_34722527_individual_individual_5_Juliana_Maria_Ferraz_Sallum1_3_Clinicalandmo
Expected at least one variant per patient, but received none for patient PMID_34722527_individual_individual_5_Juliana_Maria_Ferraz_Sallum1_3_Clinicalandmo
Patient individual_individual_5_Juliana_Maria_Ferraz_Sallum1_3_Clinicalandmo has no variants listed and will not be included in this analysis.
Patients Created:  16%|█▌        | 36/229 [00:01<00:10, 18.37it/s]Expected a VCF record, a VRS CNV, or an expression with `hgvs.c` but did not find one in patient PMID_34722527_individual_individual_3_Juliana_Maria_Ferraz_Sallum1_3_Clinicalandmo
Expected at least one variant per patient, but received none for patient PMID_34722527_individual_individual_3_Juliana_Maria_Ferraz_Sallum1_3_Clinicalandmo
Patient individual_individual_3_Juliana_Maria_Ferraz_Sallum1_3

### Pick transcript

We choose the [MANE Select](https://www.ncbi.nlm.nih.gov/nuccore/NM_020366.4) transcript for *RPGRIP1*.

In [4]:
tx_id = 'NM_020366.4'

## Explore cohort

Explore the cohort to guide selection of the genotype-phenotype analysis.


In [5]:
cohort.list_all_variants()

[('14_21312457_21312458_GA_G', 25),
 ('14_21325943_21325943_G_T', 12),
 ('14_21302530_21302531_AG_A', 8),
 ('14_21345145_21345145_C_T', 8),
 ('14_21345139_21345146_CAAGGCCG_C', 7),
 ('14_21325252_21325252_G_A', 7),
 ('14_21327671_21327671_A_AT', 6),
 ('14_21317724_21317724_C_T', 5),
 ('14_21325265_21325265_A_G', 5),
 ('14_21303542_21303542_C_T', 5),
 ('14_21326131_21326131_C_T', 4),
 ('14_21327800_21327801_CT_C', 4),
 ('14_21348174_21348174_T_G', 4),
 ('14_21348210_21348214_AAAAG_A', 4),
 ('14_21326544_21327883_ATTTTTAGTAGAGATGGGATTTCTCCATGTTGGTCAGGCTGGTCTTCAACTCCCGACCTCAGGTGAACCTCCCACCTGAGCCTCCCAAAGTGCTGGGATTACAGACGTGAGCCACCGCGCCTGGCTGAACAAACTTTTTCAAGCTCTGTAATGCTGTCTAGTATCTGTCTTTACTAAAGGCCTGTTGTTTCTTAGTGCATGACTACATAGATATCTGATTATAAACTGAGACCTTAACACTCCCCCATCATTCTCTCACTTCTTTTAAACACTGGACACAAGTTAGAGAGATTTCCACACCAGATCATGACAAACACAAATTTCTTGGATTTTTTTTTTCCTCCCAATGTGGAGCTGAGCTCCATACTGTCTTTCCTAACTTTTATACCTAGGATTGTGGGGGTGTACCAAGAGGGGTCAACTCTTTGACTACAGTCCTGGGAGGGTGAGGTGGGGGTATCCATGTTTTCCTTAGGAAGTGGG

In [6]:
cohort.list_data_by_tx()

{'NM_001377523.1': Counter({'INTRON_VARIANT': 85,
          'STOP_GAINED': 37,
          'FRAMESHIFT_VARIANT': 36,
          'SPLICE_ACCEPTOR_VARIANT': 10,
          'SPLICE_POLYPYRIMIDINE_TRACT_VARIANT': 1,
          'MISSENSE_VARIANT': 7,
          'SPLICE_REGION_VARIANT': 11,
          'SPLICE_DONOR_VARIANT': 8,
          'SPLICE_DONOR_5TH_BASE_VARIANT': 4,
          'CODING_SEQUENCE_VARIANT': 5,
          'INFRAME_DELETION': 1,
          'SYNONYMOUS_VARIANT': 1}),
 'NM_020366.4': Counter({'FRAMESHIFT_VARIANT': 95,
          'STOP_GAINED': 81,
          'MISSENSE_VARIANT': 50,
          'SPLICE_ACCEPTOR_VARIANT': 12,
          'SPLICE_POLYPYRIMIDINE_TRACT_VARIANT': 2,
          'INTRON_VARIANT': 10,
          'SPLICE_REGION_VARIANT': 20,
          'CODING_SEQUENCE_VARIANT': 7,
          'SPLICE_DONOR_VARIANT': 19,
          'SPLICE_DONOR_5TH_BASE_VARIANT': 5,
          'INFRAME_DELETION': 2,
          'SYNONYMOUS_VARIANT': 1}),
 'NM_001377948.1': Counter({'FRAMESHIFT_VARIANT': 47,
 

## Configure the analysis

In [7]:
from genophenocorr.analysis import configure_cohort_analysis
from genophenocorr.analysis.predicate import BooleanPredicate

analysis = configure_cohort_analysis(cohort, hpo)

## Run the analyses

Test for presence of genotype-phenotype correlations between missense variants vs. others.

In [8]:
from genophenocorr.model import VariantEffect

by_missense = analysis.compare_by_variant_effect(VariantEffect.MISSENSE_VARIANT, tx_id=tx_id)
by_missense.summarize(hpo, BooleanPredicate.YES)

MISSENSE_VARIANT on NM_020366.4                       No               Yes  \
                                                   Count     Percent Count   
Eye poking [HP:0001483]                               25   64.102564     3   
Very low visual acuity [HP:0032122]                   66   81.481481    15   
Reduced visual acuity [HP:0007663]                    90   78.947368    24   
Visual impairment [HP:0000505]                        90   78.947368    24   
Abnormality of vision [HP:0000504]                    90   78.947368    24   
Abnormal eye physiology [HP:0012373]                  95   79.831933    24   
Abnormality of the eye [HP:0000478]                   95   79.831933    24   
Phenotypic abnormality [HP:0000118]                   97   80.165289    24   
All [HP:0000001]                                      97   80.165289    24   
Neurodevelopmental delay [HP:0012758]                 11  100.000000     0   
Neurodevelopmental abnormality [HP:0012759]           11  100.000000     0   
Abnormal nervous system physiology [HP:0012638]       32   91.428571     3   
Abnormality of the nervous system [HP:0000707]        32   91.428571     3   
Self-injurious behavior [HP:0100716]                  25   89.285714     3   
Abnormal movements of face and head [HP:5200017]      25   89.285714     3   
Tactile sensory seeking [HP:5200065]                  25   89.285714     3   
Recurrent maladaptive behavior [HP:5200241]           25   89.285714     3   
Abnormal repetitive mannerisms [HP:0000733]           25   89.285714     3   
Sensory seeking [HP:4000079]                          25   89.285714     3   
Atypical behavior [HP:0000708]                        25   89.285714     3   
Restricted or repetitive behaviors or interests...    25   89.285714     3   
Sensory behavioral abnormality [HP:5200046]           25   89.285714     3   
Abnormality of mental function [HP:0011446]           25   89.285714     3   
High hypermetropia [HP:0008499]                        9  100.000000     0   
Hypermetropia [HP:0000540]                            20   83.333333     4   
Abnormality of refraction [HP:0000539]                24   85.714286     4   
Abnormal electroretinogram [HP:0000512]               23   82.142857     4   
Abnormal visual electrophysiology [HP:0030453]        23   85.185185     4   
High myopia [HP:0011003]                               4  100.000000     0   
Myopia [HP:0000545]                                    4  100.000000     0   
Constriction of peripheral visual field [HP:000...     9   90.000000     1   
Visual field defect [HP:0001123]                       9   90.000000     1   
Severely reduced visual acuity [HP:0001141]            6  100.000000     0   
Mild hypermetropia [HP:0031728]                        1   50.000000     1   
Moderately reduced visual acuity [HP:0030515]         15   62.500000     9   
Mildly reduced visual acuity [HP:0032037]              3  100.000000     0   
Moderate hypermetropia [HP:0031729]                    9   75.000000     3   

MISSENSE_VARIANT on NM_020366.4                                          \
                                                      Percent   p value   
Eye poking [HP:0001483]                              7.692308  0.608819   
Very low visual acuity [HP:0032122]                 18.518519  1.000000   
Reduced visual acuity [HP:0007663]                  21.052632  1.000000   
Visual impairment [HP:0000505]                      21.052632  1.000000   
Abnormality of vision [HP:0000504]                  21.052632  1.000000   
Abnormal eye physiology [HP:0012373]                20.168067  1.000000   
Abnormality of the eye [HP:0000478]                 20.168067  1.000000   
Phenotypic abnormality [HP:0000118]                 19.834711  1.000000   
All [HP:0000001]                                    19.834711  1.000000   
Neurodevelopmental delay [HP:0012758]                0.000000  1.000000   
Neurodevelopmental abnormality [HP:0012759]          0.00

Test for presence of genotype-phenotype correlations between frameshift variants vs. others.

In [9]:
by_frameshift = analysis.compare_by_variant_effect(VariantEffect.FRAMESHIFT_VARIANT, tx_id=tx_id)
by_frameshift.summarize(hpo, BooleanPredicate.YES)

FRAMESHIFT_VARIANT on NM_020366.4                     No              Yes  \
                                                   Count    Percent Count   
Eye poking [HP:0001483]                                5  12.820513    23   
Very low visual acuity [HP:0032122]                   45  55.555556    36   
Reduced visual acuity [HP:0007663]                    66  57.894737    48   
Visual impairment [HP:0000505]                        66  57.894737    48   
Abnormality of vision [HP:0000504]                    66  57.894737    48   
Abnormal eye physiology [HP:0012373]                  67  56.302521    52   
Abnormality of the eye [HP:0000478]                   67  56.302521    52   
Phenotypic abnormality [HP:0000118]                   69  57.024793    52   
All [HP:0000001]                                      69  57.024793    52   
Neurodevelopmental delay [HP:0012758]                  3  27.272727     8   
Neurodevelopmental abnormality [HP:0012759]            3  27.272727     8   
Abnormal nervous system physiology [HP:0012638]        8  22.857143    27   
Abnormality of the nervous system [HP:0000707]         8  22.857143    27   
Self-injurious behavior [HP:0100716]                   5  17.857143    23   
Abnormal movements of face and head [HP:5200017]       5  17.857143    23   
Tactile sensory seeking [HP:5200065]                   5  17.857143    23   
Recurrent maladaptive behavior [HP:5200241]            5  17.857143    23   
Abnormal repetitive mannerisms [HP:0000733]            5  17.857143    23   
Sensory seeking [HP:4000079]                           5  17.857143    23   
Atypical behavior [HP:0000708]                         5  17.857143    23   
Restricted or repetitive behaviors or interests...     5  17.857143    23   
Sensory behavioral abnormality [HP:5200046]            5  17.857143    23   
Abnormality of mental function [HP:0011446]            5  17.857143    23   
High hypermetropia [HP:0008499]                        5  55.555556     4   
Hypermetropia [HP:0000540]                            12  50.000000    12   
Abnormality of refraction [HP:0000539]                12  42.857143    16   
Abnormal electroretinogram [HP:0000512]               16  57.142857    11   
Abnormal visual electrophysiology [HP:0030453]        16  59.259259    11   
High myopia [HP:0011003]                               0   0.000000     4   
Myopia [HP:0000545]                                    0   0.000000     4   
Constriction of peripheral visual field [HP:000...     5  50.000000     5   
Visual field defect [HP:0001123]                       5  50.000000     5   
Severely reduced visual acuity [HP:0001141]            3  50.000000     3   
Mild hypermetropia [HP:0031728]                        1  50.000000     1   
Moderately reduced visual acuity [HP:0030515]         16  66.666667     8   
Mildly reduced visual acuity [HP:0032037]              2  66.666667     1   
Moderate hypermetropia [HP:0031729]                    5  41.666667     7   

FRAMESHIFT_VARIANT on NM_020366.4                                         \
                                                       Percent   p value   
Eye poking [HP:0001483]                              58.974359  0.016983   
Very low visual acuity [HP:0032122]                  44.444444  1.000000   
Reduced visual acuity [HP:0007663]                   42.105263  1.000000   
Visual impairment [HP:0000505]                       42.105263  1.000000   
Abnormality of vision [HP:0000504]                   42.105263  1.000000   
Abnormal eye physiology [HP:0012373]                 43.697479  1.000000   
Abnormality of the eye [HP:0000478]                  43.697479  1.000000   
Phenotypic abnormality [HP:0000118]                  42.975207  1.000000   
All [HP:0000001]                                     42.975207  1.000000   
Neurodevelopmental delay [HP:0012758]                72.727273  1.000000   
Neurodevelopmental abnormality [HP:0012759]          72.727273  1.000000   
Abnormal

Or between subjects with >=1 allele of a variant vs. the other subjects:

In [10]:
variant_key = '14_21312457_21312458_GA_G'

by_var = analysis.compare_by_variant_key(variant_key)
by_var.summarize(hpo, BooleanPredicate.YES)

>=1 allele of the variant 14_21312457_21312458_GA_G    No               Yes  \
                                                    Count     Percent Count   
Eye poking [HP:0001483]                                12   30.769231    16   
Very low visual acuity [HP:0032122]                    64   79.012346    17   
Reduced visual acuity [HP:0007663]                     97   85.087719    17   
Visual impairment [HP:0000505]                         97   85.087719    17   
Abnormality of vision [HP:0000504]                     97   85.087719    17   
Abnormal eye physiology [HP:0012373]                  102   85.714286    17   
Abnormality of the eye [HP:0000478]                   102   85.714286    17   
Phenotypic abnormality [HP:0000118]                   104   85.950413    17   
All [HP:0000001]                                      104   85.950413    17   
Neurodevelopmental delay [HP:0012758]                   7   63.636364     4   
Neurodevelopmental abnormality [HP:0012759]             7   63.636364     4   
Abnormal nervous system physiology [HP:0012638]        19   54.285714    16   
Abnormality of the nervous system [HP:0000707]         19   54.285714    16   
Self-injurious behavior [HP:0100716]                   12   42.857143    16   
Abnormal movements of face and head [HP:5200017]       12   42.857143    16   
Tactile sensory seeking [HP:5200065]                   12   42.857143    16   
Recurrent maladaptive behavior [HP:5200241]            12   42.857143    16   
Abnormal repetitive mannerisms [HP:0000733]            12   42.857143    16   
Sensory seeking [HP:4000079]                           12   42.857143    16   
Atypical behavior [HP:0000708]                         12   42.857143    16   
Restricted or repetitive behaviors or interests...     12   42.857143    16   
Sensory behavioral abnormality [HP:5200046]            12   42.857143    16   
Abnormality of mental function [HP:0011446]            12   42.857143    16   
High hypermetropia [HP:0008499]                         6   66.666667     3   
Hypermetropia [HP:0000540]                             21   87.500000     3   
Abnormality of refraction [HP:0000539]                 25   89.285714     3   
Abnormal electroretinogram [HP:0000512]                27   96.428571     0   
Abnormal visual electrophysiology [HP:0030453]         27  100.000000     0   
High myopia [HP:0011003]                                4  100.000000     0   
Myopia [HP:0000545]                                     4  100.000000     0   
Constriction of peripheral visual field [HP:000...     10  100.000000     0   
Visual field defect [HP:0001123]                       10  100.000000     0   
Severely reduced visual acuity [HP:0001141]             6  100.000000     0   
Mild hypermetropia [HP:0031728]                         2  100.000000     0   
Moderately reduced visual acuity [HP:0030515]          24  100.000000     0   
Mildly reduced visual acuity [HP:0032037]               3  100.000000     0   
Moderate hypermetropia [HP:0031729]                    12  100.000000     0   

>=1 allele of the variant 14_21312457_21312458_GA_G                       \
                                                       Percent   p value   
Eye poking [HP:0001483]                              41.025641  0.000919   
Very low visual acuity [HP:0032122]                  20.987654  1.000000   
Reduced visual acuity [HP:0007663]                   14.912281  1.000000   
Visual impairment [HP:0000505]                       14.912281  1.000000   
Abnormality of vision [HP:0000504]                   14.912281  1.000000   
Abnormal eye physiology [HP:0012373]                 14.285714  1.000000   
Abnormality of the eye [HP:0000478]                  14.285714  1.000000   
Phenotypic abnormality [HP:0000118]                  14.049587  1.000000   
All [HP:0000001]                                     14.049587  1.000000   
Neurodevelopmental delay [HP:0012758]                36.363636  1.000000   
Neurod

TODO - finalize!